In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Obtener la lista de archivos JSON en carpeta
#archivos_json = glob.glob(os.path.join("datos/yelp", "*.json"))

In [4]:
# Se guardará archivos JSON en formato pickle (binario) el cual es más rápido para futuras consultas
# for archivo in archivos_json:

archivo = 'datos/yelp_yelp/yelp_academic_dataset_review.json'
df = pd.read_json(archivo, lines=True)
df.to_pickle(archivo.replace(".json", ".pkl"))  # Guardar como .pkl
business = pd.read_pickle("datos/yelp_yelp/yelp_academic_dataset_review.pkl")

business.head()

MemoryError: 

In [5]:
archivo = 'datos/yelp_yelp/yelp_academic_dataset_review.json'
chunks = pd.read_json(archivo, lines=True, chunksize=100000)  # Leer en bloques de 100,000 filas

# Guardar cada chunk en un archivo pickle
# Esto evita que la memoria se llene al procesar el JSON por partes.
for i, chunk in enumerate(chunks):
    chunk.to_pickle(f'datos/yelp_yelp/review_part_{i}.pkl')  # Guarda en archivos más pequeños
    print(f'Chunk {i} guardado.')

Chunk 0 guardado.
Chunk 1 guardado.
Chunk 2 guardado.
Chunk 3 guardado.
Chunk 4 guardado.
Chunk 5 guardado.
Chunk 6 guardado.
Chunk 7 guardado.
Chunk 8 guardado.
Chunk 9 guardado.
Chunk 10 guardado.
Chunk 11 guardado.
Chunk 12 guardado.
Chunk 13 guardado.
Chunk 14 guardado.
Chunk 15 guardado.
Chunk 16 guardado.
Chunk 17 guardado.
Chunk 18 guardado.
Chunk 19 guardado.
Chunk 20 guardado.
Chunk 21 guardado.
Chunk 22 guardado.
Chunk 23 guardado.
Chunk 24 guardado.
Chunk 25 guardado.
Chunk 26 guardado.
Chunk 27 guardado.
Chunk 28 guardado.
Chunk 29 guardado.
Chunk 30 guardado.
Chunk 31 guardado.
Chunk 32 guardado.
Chunk 33 guardado.
Chunk 34 guardado.
Chunk 35 guardado.
Chunk 36 guardado.
Chunk 37 guardado.
Chunk 38 guardado.
Chunk 39 guardado.
Chunk 40 guardado.
Chunk 41 guardado.
Chunk 42 guardado.
Chunk 43 guardado.
Chunk 44 guardado.
Chunk 45 guardado.
Chunk 46 guardado.
Chunk 47 guardado.
Chunk 48 guardado.
Chunk 49 guardado.
Chunk 50 guardado.
Chunk 51 guardado.
Chunk 52 guardado.
Chu

In [6]:
import glob

# Obtener lista de archivos .pkl guardados
archivos_pkl = glob.glob('datos/yelp_yelp/review_part_*.pkl')

# Leer y concatenar los DataFrames
df_reviews = pd.concat([pd.read_pickle(f) for f in archivos_pkl])

print(df_reviews.head())

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm... 2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delicious.

In [7]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6990280 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 533.3+ MB


In [8]:
print('text NaN', df_reviews['text'].isna().sum())
print('stars NaN', df_reviews['stars'].isna().sum())

text NaN 0
stars NaN 0


In [ ]:
# Convertir la columna a tipo datetime
# Vamos a asegurarnos de que la columna "date" esté en el formato correcto:
df_reviews['date'] = pd.to_datetime(df_reviews['date'], errors='coerce')

In [9]:
# df_rev.to_parquet('datos/yelp/reviews.parquet' , engine='fastparquet')
# El motor pyarrow maneja mejor archivos grandes y evita problemas de compresión:
df_reviews.to_parquet('datos/yelp/reviews.parquet', engine='pyarrow', compression='snappy')

In [3]:
# leo el archivo de las reviews de yelp
# How to read a Parquet file into Pandas DataFrame?
df_rev = pd.read_parquet('datos/yelp/reviews.parquet' , engine='pyarrow')

In [10]:
df_dup = df_reviews[df_reviews.duplicated(subset=['business_id',  'date', 'review_id', 'user_id'], keep=False)]
df_dup = df_reviews.sort_values([])

In [11]:
print(df_dup)

                     review_id                 user_id  \
0       KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA   
1       BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q   
2       saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A   
3       AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ   
4       Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ   
...                        ...                     ...   
999995  t-2o35kr7Q9DSaeuKhaDuQ  oX7o1TH0PHUWp9r9ry9_vw   
999996  fLIwWCvdul9PNWYfJt5QWA  v8wlapFKVLs2qTYCGhCdiw   
999997  ETAiy6wEM-r9ve4SKDhBpg  rLlYc1RzIBnOmnX3AbpEYw   
999998  8OgvSXuc6KjAt2fSz9LuzA  eEH-8CEPU5ndPxDGzVfHiQ   
999999  TQoVzNXqDkiDXgzlw8cyaQ  HYmGwYXvcYmW7dDjuWKJfw   

                   business_id  stars  useful  funny  cool  \
0       XQfwVwDr-v0ZS3_CbbE5Xw      3       0      0     0   
1       7ATYjTIgM3jUlt4UM3IypQ      5       1      0     1   
2       YjUWPpI6HXG530lwP-fb2A      3       0      0     0   
3       kxX2SOes4o-D3ZQBkiMRfA      5       1      0   

In [12]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_sindup = df_reviews.drop_duplicates(subset=['business_id',  'date', 'review_id', 'user_id'], keep='first')

In [13]:
# leo el archivo de las pizzerias mt_pizza_NJNY para hacer join con este
# How to read a Parquet file into Pandas DataFrame?
business_pizza_NJNY = pd.read_parquet('datos/yelp_yelp/business_pizza_NJNY.parquet' , engine='fastparquet')

In [14]:
business_pizza_NJNY.info()

<class 'pandas.core.frame.DataFrame'>
Index: 717 entries, 764 to 150236
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   717 non-null    object 
 1   name          717 non-null    object 
 2   address       717 non-null    object 
 3   city          717 non-null    object 
 4   state         717 non-null    object 
 5   postal_code   717 non-null    object 
 6   latitude      717 non-null    float64
 7   longitude     717 non-null    float64
 8   stars         717 non-null    float64
 9   review_count  717 non-null    int64  
 10  categories    717 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 67.2+ KB


In [15]:
# creo que no hay columnas para eliminar. Guardo el archivo de reviews sin dup
df_rev_sindup.to_parquet('datos/yelp_yelp/rev_sindup.parquet', engine='pyarrow', compression='snappy')

In [16]:
# cuento la cantidad de pizzerias en cada estado / nopuedo contar con una sola columna
business_pizza_NJNY[['state', 'name' ]].groupby('state').count()

,name
state,
NJ,717


In [17]:
business_pizza_NJ_rev = pd.merge(business_pizza_NJNY, df_rev_sindup, how='left', on='business_id') # tengo que hacer left join

In [18]:
business_pizza_NJ_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28766 entries, 0 to 28765
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   business_id   28766 non-null  object        
 1   name          28766 non-null  object        
 2   address       28766 non-null  object        
 3   city          28766 non-null  object        
 4   state         28766 non-null  object        
 5   postal_code   28766 non-null  object        
 6   latitude      28766 non-null  float64       
 7   longitude     28766 non-null  float64       
 8   stars_x       28766 non-null  float64       
 9   review_count  28766 non-null  int64         
 10  categories    28766 non-null  object        
 11  review_id     28766 non-null  object        
 12  user_id       28766 non-null  object        
 13  stars_y       28766 non-null  int64         
 14  useful        28766 non-null  int64         
 15  funny         28766 non-null  int64 

In [19]:
# rename column name to user_name
business_pizza_NJ_rev.rename(columns={'stars_x': 'stars_business', 'stars_y': 'stars_by_user'}, inplace=True)

In [20]:
business_pizza_NJ_rev.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_business,review_count,categories,review_id,user_id,stars_by_user,useful,funny,cool,text,date
0,Mt1FB8hM4XTi3Tk4i0q5gQ,Nicola's Pizza,8 N Franklin St,Lambertville,NJ,08530,40.366418,-74.941678,4.0,54,"Restaurants, Pizza, Salad",SunZmQqZ51UeOMwlKoupUQ,6jUggSkt0X01p2JytyMfjw,5,0,0,0,It's a great place close to new hope across th...,2015-10-24 22:24:32
1,Mt1FB8hM4XTi3Tk4i0q5gQ,Nicola's Pizza,8 N Franklin St,Lambertville,NJ,08530,40.366418,-74.941678,4.0,54,"Restaurants, Pizza, Salad",HHEr1cX_mxrFoq2vGJwHyg,Ad20gaFq7_wy7BrOf8WXsQ,4,0,0,0,Great Jersey pizza here....thin with a great c...,2018-04-05 18:40:53
2,Mt1FB8hM4XTi3Tk4i0q5gQ,Nicola's Pizza,8 N Franklin St,Lambertville,NJ,08530,40.366418,-74.941678,4.0,54,"Restaurants, Pizza, Salad",GmZBkO3re-zWFY0JFXCsBQ,PGkMTm3hiermfR6TE498bw,4,0,0,0,I would say that this is the best pizza I've h...,2013-11-16 22:54:48
3,Mt1FB8hM4XTi3Tk4i0q5gQ,Nicola's Pizza,8 N Franklin St,Lambertville,NJ,08530,40.366418,-74.941678,4.0,54,"Restaurants, Pizza, Salad",HNBUtHDGBJRNSlu3I3AuQw,bdtwToDfakvK7pRr6eyQxA,5,1,1,0,may just be the best pizza i've ever had. my h...,2016-10-27 10:13:57
4,Mt1FB8hM4XTi3Tk4i0q5gQ,Nicola's Pizza,8 N Franklin St,Lambertville,NJ,08530,40.366418,-74.941678,4.0,54,"Restaurants, Pizza, Salad",3ZgbLNfqjKX9Ea61AL7vYg,2DHpnFd1pdmRZD8TaDpXaw,5,0,0,0,Love Nicola's! The pizza is consistently great...,2015-05-02 23:13:51


In [21]:
print(business_pizza_NJ_rev.iloc[[0]])

              business_id            name          address          city  \
0  Mt1FB8hM4XTi3Tk4i0q5gQ  Nicola's Pizza  8 N Franklin St  Lambertville   

  state postal_code   latitude  longitude  stars_business  review_count  \
0    NJ       08530  40.366418 -74.941678             4.0            54   

                  categories               review_id                 user_id  \
0  Restaurants, Pizza, Salad  SunZmQqZ51UeOMwlKoupUQ  6jUggSkt0X01p2JytyMfjw   

   stars_by_user  useful  funny  cool  \
0              5       0      0     0   

                                                text                date  
0  It's a great place close to new hope across th... 2015-10-24 22:24:32  


In [22]:
# quiero contar la cantidad de reseñas que tiene cada negocio 
df_reviews_count = business_pizza_NJ_rev.groupby(['business_id', 'name', 'address',  'city', 'state', 'postal_code' , 'review_count'  ]).size().reset_index(name='num_reviews')

In [23]:
# ordena los negocios por número de reseñas de mayor a menor.
df_reviews_count = df_reviews_count.sort_values(by='num_reviews', ascending=False)

In [24]:
# guardo df_pizza_NJNY_rev
business_pizza_NJ_rev.to_parquet('datos/yelp_yelp/business_pizza_NJ_rev.parquet' , engine='fastparquet')

In [25]:
df_reviews_count.to_excel("datos/yelp_yelp/business_pizza_NJ_rev_reviews_counts.xlsx")

In [2]:
# leo business_pizza_NJ_rev
# How to read a Parquet file into Pandas DataFrame?
business_pizza_NJ_rev = pd.read_parquet('datos/yelp_yelp/reviews.parquet' , engine='fastparquet')

In [4]:
# Buscamos NaN en columna 'user_id' 
# serian los locales que no tienen reviews
business_pizza_NJ_rev['review_id'].isna().sum()

np.int64(0)

In [5]:
business_pizza_NJ_rev[business_pizza_NJ_rev['user_id'].isna()]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
